In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/UBC-OCEAN/sample_submission.csv
/kaggle/input/UBC-OCEAN/train.csv
/kaggle/input/UBC-OCEAN/test.csv
/kaggle/input/UBC-OCEAN/test_thumbnails/41_thumbnail.png
/kaggle/input/UBC-OCEAN/train_images/14127.png
/kaggle/input/UBC-OCEAN/train_images/34649.png
/kaggle/input/UBC-OCEAN/train_images/15221.png
/kaggle/input/UBC-OCEAN/train_images/52375.png
/kaggle/input/UBC-OCEAN/train_images/17487.png
/kaggle/input/UBC-OCEAN/train_images/32112.png
/kaggle/input/UBC-OCEAN/train_images/22290.png
/kaggle/input/UBC-OCEAN/train_images/48734.png
/kaggle/input/UBC-OCEAN/train_images/39146.png
/kaggle/input/UBC-OCEAN/train_images/64950.png
/kaggle/input/UBC-OCEAN/train_images/32042.png
/kaggle/input/UBC-OCEAN/train_images/63429.png
/kaggle/input/UBC-OCEAN/train_images/13526.png
/kaggle/input/UBC-OCEAN/train_images/53859.png
/kaggle/input/UBC-OCEAN/train_images/63836.png
/kaggle/input/UBC-OCEAN/train_images/56117.png
/kaggle/input/UBC-OCEAN/train_images/7490.png
/kaggle/input/UBC-OCEAN/train_im

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
import os
import seaborn as sns
import cv2
import random
import os
import glob
import torch
import torch.nn as nn

In [15]:
pd.read_csv('/kaggle/input/UBC-OCEAN/train.csv').shape

(538, 5)

In [7]:
ROOT_DIR = '/kaggle/input/UBC-OCEAN'
TRAIN_DIR = '/kaggle/input/UBC-OCEAN/train_thumbnails'
TEST_DIR = '/kaggle/input/UBC-OCEAN/test_thumbnails'


def get_train_file_path(image_id):
    return f"{TRAIN_DIR}/{image_id}_thumbnail.png"

In [16]:
# Assuming you have a DataFrame `df` containing a 'file_path' column with the paths to the images
df = pd.read_csv(f"{ROOT_DIR}/train.csv")
df['file_path'] = df['image_id'].apply(get_train_file_path)

# Get unique labels and their counts
print("Unique Labels and Counts:")
print(df['label'].value_counts())

# Read a few images and print their dimensions
sample_images = df[df['is_tma'] == False]['file_path'].sample(5)  # just a random sample of 5 images
for img_path in sample_images:
    img = cv2.imread(img_path)
    print(f"Dimensions for image at {img_path}: ", img.shape)


Unique Labels and Counts:
label
HGSC    222
EC      124
CC       99
LGSC     47
MC       46
Name: count, dtype: int64
Dimensions for image at /kaggle/input/UBC-OCEAN/train_thumbnails/39880_thumbnail.png:  (2102, 3000, 3)
Dimensions for image at /kaggle/input/UBC-OCEAN/train_thumbnails/24507_thumbnail.png:  (3149, 3000, 3)
Dimensions for image at /kaggle/input/UBC-OCEAN/train_thumbnails/4_thumbnail.png:  (2523, 3000, 3)
Dimensions for image at /kaggle/input/UBC-OCEAN/train_thumbnails/25792_thumbnail.png:  (1550, 3000, 3)
Dimensions for image at /kaggle/input/UBC-OCEAN/train_thumbnails/8985_thumbnail.png:  (1465, 3000, 3)


In [17]:

# # Your class distribution turned into weights (step 1 and 2)
# weights = torch.tensor([1/0.413, 1/0.086, 1/0.087, 1/0.184, 1/0.23])
# weights = weights / weights.sum()

# # Initialize the loss function with the weights
# criterion = nn.CrossEntropyLoss(weight=weights.to(device))

# # During training, use this loss function
# # ...
# output = model(input_data)
# loss = criterion(output, labels)
# # ...


Class distribution:
 label
HGSC    222
EC      124
CC       99
LGSC     47
MC       46
Name: count, dtype: int64
